<div style="font-size:30pt; font-weight:700;margin-top:50px;margin-bottom:50px;color:royalblue; text-align:center;width:800px;line-height:20pt">Kaggle New York City Taxi Fare Prediction</div>

This workbook is the New York City Taxi Fare Prediction dataset analysis to predict the fare of a taxi trip in New York City.

It is a Kaggle competition available <a href="https://www.kaggle.com/c/new-york-city-taxi-fare-prediction" target="_blank">HERE</a>.

<i><blockquote><b>The Challenge</b></blockquote></i>
    
<blockquote>In this playground competition, hosted in partnership with Google Cloud and Coursera, you are tasked with predicting the fare amount (inclusive of tolls) for a taxi ride in New York City given the pickup and dropoff locations. While you can get a basic estimate based on just the distance between the two points, this will result in an RMSE of $5-$8, depending on the model used (see the starter code for an example of this approach in Kernels). Your challenge is to do better than this using Machine Learning techniques!

To learn how to handle large datasets with ease and solve this problem using TensorFlow, consider taking the Machine Learning with TensorFlow on Google Cloud Platform specialization on Coursera -- the taxi fare problem is one of several real-world problems that are used as case studies in the series of courses. To make this easier, head to Coursera.org/NEXTextended to claim this specialization for free for the first month!</blockquote>
<figcaption>Kaggle</figcaption>

# 1. Import

In [ ]:
#
from scipy import stats
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight');
plt.rcParams['font.size'] = 14;
plt.figure(figsize=(12,5));
palette = sns.color_palette('Paired', 10);

# map
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

# Sci-kit learn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn import set_config; set_config(display='diagram')

In [ ]:
data = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/train.csv", nrows=1000000)
data.head()

# 2. Dropping useless features

In [ ]:
data = data.drop(["key"], axis=1)

# 3. Duplicates

In [ ]:
size_before = len(data)
data = data.drop_duplicates()
size_after = len(data)
print(str(size_before - size_after) + " duplicates were removed.")

# 4. Missing
<div style="font-weight:700">Droping features that have too many missing values</div>

In [ ]:
100 * data.isnull().sum().sort_values(ascending=False)/len(data)

<div style="font-weight:700">Seems there is nothing to drop.</div>

# 5. Explore Data

In [ ]:
def plot_dist(series=data["fare_amount"], title="Fare Distribution"):
    sns.histplot(series, kde=True, stat='density', discrete=True)
    sns.despine()
    plt.title(title);
    plt.show()
plot_dist()

<div style="font-weight:700">Let's drop absurd values.</div>

In [ ]:
data = data[data.fare_amount.between(0, 60)]
plot_dist(data.fare_amount)

<div style="font-weight:700">We can also visualise binned fare_amount variable:</div>

In [ ]:
data['fare-bin'] = pd.cut(data['fare_amount'], bins = list(range(0, 50, 5)), include_lowest=True).astype('str')

# Uppermost bin
data['fare-bin'] = data['fare-bin'].replace(np.nan, '[45+]')

# apply this to clean up the first bin's label
data['fare-bin'] = data['fare-bin'].apply(lambda x: x.replace('-0.001', '0'))

# sort by fare the correct look in the chart
data = data.sort_values(by='fare_amount')

In [ ]:
sns.catplot(x="fare-bin", kind="count", palette=palette, data=data, height=5, aspect=3);
sns.despine()
plt.show()

<div style="font-weight:700">Let's have a closer look at <i style="color:royalblue">passenger_count</i> feature</div>

In [ ]:
data.passenger_count.describe()

In [ ]:
sns.catplot(x="passenger_count", kind="count", palette=palette, data=data, height=5, aspect=3);
sns.despine()
plt.title('Passenger Count');
plt.show()

<div style="font-weight:700">Let's have a look at <i style="color:royalblue">pickup_datetime</i> feature</div>

In [ ]:
def extract_time_features(df):
    timezone_name = 'America/New_York'
    time_column = "pickup_datetime"
    df.index = pd.to_datetime(df[time_column])
    df.index = df.index.tz_convert(timezone_name)
    df["dow"] = df.index.weekday
    df["hour"] = df.index.hour
    df["month"] = df.index.month
    df["year"] = df.index.year
    return df.reset_index(drop=True)

In [ ]:
data = extract_time_features(data.drop(["fare-bin"], axis=1))
data.head()

<div style="font-weight:700">Taxi trip repartition by hour of the day</div>

In [ ]:
sns.catplot(x="hour", kind="count", palette=palette, data=data, height=5, aspect=3);
sns.despine()
plt.title('Hour of Day');
plt.show()

<div style="font-weight:700">Taxi trip repartition by day of week</div>

In [ ]:
sns.catplot(x="dow", kind="count", palette=palette, data=data, height=5, aspect=3);
sns.despine()
plt.title('Day of Week');
plt.show()

<div style="font-weight:700">Let's have a look at <span style="color:royalblue;font-variant:small-caps">geospatial</span> features</div>
<br/>
<div style="font-weight:700">Find boudaries from test set and remove outliers from training set</div>

In [ ]:
data_test = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/test.csv")

In [ ]:
for col in ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]:
    MIN = data_test[col].min()
    MAX = data_test[col].max()
    print(col, MIN, MAX)

In [ ]:
data = data[data["pickup_latitude"].between(left = 40, right = 42 )]
data = data[data["pickup_longitude"].between(left = -74.3, right = -72.9 )]
data = data[data["dropoff_latitude"].between(left = 40, right = 42 )]
data = data[data["dropoff_longitude"].between(left = -74, right = -72.9 )]

In [ ]:
center_location = [40.758896, -73.985130]
m = folium.Map(location=center_location, control_scale=True, zoom_start=11)

In [ ]:
data["count"] =1
heatmap_data = data.head(10000)[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist()
gradient = {0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}
HeatMap(data=heatmap_data, radius=5, gradient=gradient, max_zoom=13).add_to(m)
m

In [ ]:
heatmap_data_by_hour = []
__data__ = data.head(10000)
for hour in data.hour.sort_values().unique():
    _data = __data__[__data__.hour == hour][['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist()
    heatmap_data_by_hour.append(_data)

In [ ]:
m2 = folium.Map(location=center_location, control_scale=True, zoom_start=11)
HeatMapWithTime(heatmap_data_by_hour, radius=5, 
                gradient=gradient, 
                min_opacity=0.5, max_opacity=0.8, 
                use_local_extrema=False).add_to(m2)
m2

<div style="font-weight:700">Let's have a look at <span style="color:royalblue;font-variant:small-caps">distance</span> features</div>
<br/>
<div style="font-weight:700">Here it is a method to compute distance between two point from their geospatial coordinates</div>

In [ ]:
def haversine_distance(df,
                       start_lat="start_lat",
                       start_lon="start_lon",
                       end_lat="end_lat",
                       end_lon="end_lon"):
    """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df
        Computes distance in kms
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    haversine_distance = 6371 * c
    return haversine_distance

data["distance"] = haversine_distance(data, 
                                      start_lat="pickup_latitude", start_lon="pickup_longitude",
                                      end_lat="dropoff_latitude", end_lon="dropoff_longitude"
                                     )

In [ ]:
data.distance.describe()

In [ ]:
%matplotlib inline
plot_dist(series=data[data.distance < 50].distance, title='Distance distribution')

<div style="font-weight:700">Let's have a look at <i style="color:royalblue">passenger_count</i> feature</div>

In [ ]:
sns.catplot(x="passenger_count", y="fare_amount", palette=palette, data=data, kind="bar", aspect=3)
sns.despine()
plt.show()

<div style="font-weight:700">Let's have a look at <i style="color:royalblue">fare_amount</i> feature by hour</div>

In [ ]:
sns.catplot(x="hour", y="fare_amount", palette=palette, data=data, kind="bar", aspect=3)
sns.despine()
plt.show()

<div style="font-weight:700">Let's have a look at <i style="color:royalblue">fare_amount</i> feature by day of week</div>

In [ ]:
sns.catplot(x="dow", y="fare_amount", palette=palette, data=data, kind="bar", aspect=3)
sns.despine()
plt.show()

<div style="font-weight:700">Let's have a look at the corelation between <i style="color:royalblue">fare_amount</i> and <i style="color:royalblue">distance</i> features</div>

In [ ]:
sns.scatterplot(x="distance", y="fare_amount", data=data[data.distance < 80].sample(100000))
plt.show()

In [ ]:
data.head()

# 6. Training
<div style="font-weight:700">Starting on a fresh dataset to prepare for training</div>

In [ ]:
data_train = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/train.csv", nrows=1000)

## 6.1. Cleaning dataset

In [ ]:
def clean_data(df, test=False, predict=False):
    df = df.drop(["key"], axis=1)
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 0]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df
data_train = clean_data(data_train)
data_train.head()

## 6.2. Preparing model inputs

In [ ]:
X_train = data_train.drop(["fare_amount"], axis=1)
y_train = data_train["fare_amount"]

## 6.3. Pipelines

### 6.3.1. Time features

#### 6.3.1.1 Class for time features encoding

In [ ]:
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """Extract the day of week (dow), the hour, the month and the year from a time column."""

    def __init__(self, time_column, time_zone_name='America/New_York'):
        self.time_column = time_column
        self.time_zone_name = time_zone_name

    def extract_time_features(self, X):
        timezone_name = self.time_zone_name
        time_column = self.time_column
        df = X.copy()
        df.index = pd.to_datetime(df[time_column])
        df.index = df.index.tz_convert(timezone_name)
        df["dow"] = df.index.weekday
        df["hour"] = df.index.hour
        df["month"] = df.index.month
        df["year"] = df.index.year        
        return df
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        """Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'"""
        return self.extract_time_features(X)[['dow', 'hour', 'month', 'year']].reset_index(drop=True)

#### 6.3.1.2. Test of the TimeFeaturesEncoder

In [ ]:
# test the TimeFeaturesEncoder
time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

#### 6.3.1.3. Pipeline for time features

In [ ]:
# TIME PIPELINE
pipe_time = Pipeline([
    ('time_features_create', TimeFeaturesEncoder('pickup_datetime')),
    ('time_features_ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
pipe_time

### 6.3.2. Distances

#### 6.3.2.1 Class for distance encoding

In [ ]:
def haversine_vectorized(df, 
         start_lat="pickup_latitude",
         start_lon="pickup_longitude",
         end_lat="dropoff_latitude",
         end_lon="dropoff_longitude"):

    """ 
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df
        Computes distance in kms
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [ ]:
class DistanceTransformer(BaseEstimator, TransformerMixin):
    """Compute the haversine distance between two GPS points."""

    def __init__(self, 
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude", 
                 end_lat="dropoff_latitude", 
                 end_lon="dropoff_longitude"):
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """Returns a copy of the DataFrame X with only one column: 'distance'"""
        return pd.DataFrame(haversine_vectorized(X)).rename(columns={0: "course distance [km]"}).copy()

#### 6.3.2.2. Test of the DistanceTransformer

In [ ]:
dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

### 6.3.3. Combination of distance and time features pipeline

In [ ]:
preprocessor = ColumnTransformer([
    ('distance', DistanceTransformer(), ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']),
    ('time', pipe_time, ['pickup_datetime'])
], remainder='passthrough')
preprocessor

### 6.3.4. Full preprocessor pipeline

In [ ]:
pipe_prepro = Pipeline([
    ('dist_and_time', preprocessor),
    ('scaler', MinMaxScaler())
])
pipe_prepro

### 6.3.5. Finally, Full pipeline. We choose RandomForestRegressor

In [ ]:
final_pipe = Pipeline([
    ('preprocessor', pipe_prepro),
    ('model', RandomForestRegressor())
])
final_pipe

## 6.4. Definition of custom score for RMSE

In [ ]:
def custom_rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_true - y_pred)))

In [ ]:
rmse = make_scorer(custom_rmse, greater_is_better=False)

## 6.5. Baseline RMSE for RandomForestRegressor

In [ ]:
baseline = cross_validate(final_pipe,
                          X_train,
                          y_train,
                          scoring=rmse,
                          cv=5)
baseline_rmse = -round(baseline["test_score"].mean(), 3)
baseline_rmse

## 6.6. RandomizedSearchCV for an optimized model

In [ ]:
grid_RFR = {'model__n_estimators': stats.randint(1, 300),
            'model__max_depth': stats.randint(1, 300),
            'model__max_samples': stats.randint(1, 300),
            "preprocessor__scaler": [StandardScaler(), RobustScaler(), MinMaxScaler()]
            }

search_RFR = RandomizedSearchCV(final_pipe,
                                grid_RFR,
                                scoring=rmse,
                                n_iter=100,
                                cv=5,
                                n_jobs=-1,
                                verbose=True)
search_RFR.fit(X_train, y_train);

In [ ]:
search_RFR.best_params_

In [ ]:
print("Tuned RandomForestRegressor model rmse: " + str(-round(search_RFR.best_score_, 2)))